In [1]:
import graph_script as gs
import pandas as pd

In [2]:
steps = gs.Steps()
graph = gs.StateGraph(gs.ProblemDecomposition, input=gs.InputProblem, output=gs.Plan)
graph.add_node("problem_decomposition", steps.intitial_problem_decomposition)
graph.add_node("self_evaluate", steps.self_evaluate)
graph.add_node("refine_tasks", steps.refine_tasks)
graph.add_node("finilize_decomposition", steps.finilize_decomposition)

graph.add_edge(gs.START, "problem_decomposition")
graph.add_edge("problem_decomposition", "self_evaluate")
graph.add_conditional_edges("self_evaluate", 
                            lambda state: "finalize" if state["decomposed"] else "refine", 
                            {"finalize": "finilize_decomposition", "refine": "refine_tasks"})
graph.add_edge("refine_tasks", "self_evaluate")
graph.add_edge("finilize_decomposition", gs.END)


graph = graph.compile()

In [3]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/google/boolq/" + splits["train"])

d:\work\Cognitive-Schema-Fitting\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df.head()

,question,answer,passage
0,do iran and afghanistan speak the same language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,True,Windows Movie Maker (formerly known as Windows...
3,is confectionary sugar the same as powdered sugar,True,"Powdered sugar, also called confectioners' sug..."
4,is elder scrolls online the same as skyrim,False,As with other games in The Elder Scrolls serie...


In [14]:
head_df = df.head()

for row in head_df.itertuples():
    print(row.question)
    for task in graph.stream({"input_problem": f"**Context:** {row.passage} **Question:** {row.question}"}):
        print(task)
        print("----------------")
    task["finilize_decomposition"]["tasks"]
    print("---DONE---")

do iran and afghanistan speak the same language
{'problem_decomposition': {'messages': [SystemMessage(content="You are the AI component of a multi-agent AI system. Your task is to decompose a given problem into smaller tasks.\n                                You always think before your answer. Always think on the language language of the user's input prompt.", additional_kwargs={}, response_metadata={}, id='196cdb89-ae8a-4c34-bb54-23ce98329f02'), HumanMessage(content="**User prompt:**\n**Context:** Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabe

In [15]:
task["finilize_decomposition"]["tasks"]

[{'name': 'Compare Game Settings and Timeline',
  'description': 'Understand and compare the settings and timelines of The Elder Scrolls Online and Skyrim.',
  'expected_output': 'A summary of the similarities and differences in the settings and timelines of both games.'},
 {'name': 'Analyze Gameplay Mechanics and Structure',
  'description': 'Identify and compare the core gameplay mechanics and structure of The Elder Scrolls Online and Skyrim.',
  'expected_output': 'A comparison of the gameplay mechanics and structures of both games.'},
 {'name': 'Evaluate Storyline and Objectives',
  'description': 'Examine and compare the main storylines and objectives of The Elder Scrolls Online and Skyrim.',
  'expected_output': 'An analysis of the narrative and objectives in both games.'},
 {'name': 'Assess Game Type and Player Experience',
  'description': 'Determine and compare the type of game and player experience offered by The Elder Scrolls Online and Skyrim.',
  'expected_output': 'A comp